In [41]:
from random import randint

class Obj_2048:
    
    directions = {'right' : (True, True), 
                 'Down': (False, True),
                 'left' : (True, False),
                 'Up': (False, False)}
    
    def reset(self):
        self._values = [0]*16
        self._mark = 0
        self._steps = 0
        
    def __init__(self):
        self.reset()
    
    def getMark(self):
        return self._mark
    
    def getSteps(self):
        return self._steps
    
    def getHighestNum(self):
        return max(self._values)
    
    # Array can be a row or a column
    # return the marks to be added
    def moveEntryInArray(array, ind):
        if (array[ind] != 0):
            for indBefore in range(ind)[::-1]:
                if (array[indBefore] != 0 and array[indBefore] == array[ind]):
                    array[indBefore] *= 2
                    array[ind] = 0
                    return array[indBefore]
        return 0
                        
    # return the marks to be added
    def moveArray(array):
        assert(len(array) == 4)
        marksToBeAdded = 0
        for ind in range(1,4):
            marksToBeAdded += moveEntryInArray(array, ind)
        return marksToBeAdded
    
    def unpackArray(self, indList, array):
        assert(len(indList) == 4)
        for i in range(4):
            self._values[indList[i]] = array[indList[i]]
    
    # We define down and right as forward direction
    def getIthArrayInd(i, isRow, isForward):
        margin = lambda isRow: 1 if isRow else 4
        direction = lambda isForward: 1 if isForward else -1
        return [i + j * margin for j in range(4)[::direction]]
         
    def move(self,direction, isCheck):
        isRow, isForward = dirctions[direction]
        marksToBeAdded = 0
        for i in range(4):
            indList = getIthArrayInd(i, isRow, isForward)
            array = [self.values[ind] for ind in indList]
            mark = moveArray(array)
            #means something changed
            if (mark != 0):
                #if it is a player's action, change the values
                if not isCheck:
                    self.unpackArray(indList,array)
                marksToBeAdded += mark
        if not isCheck:
            self._mark += marksToBeAdded
            self._step += 1
        else:
            return marksToBeAdded != 0
        
    def isGameOver(self):
        for move in directions.keys():
            if (self.move(move, True)):
                return False
        return True
            
    